# Headphones V2 - Evaluation

In [1]:
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
from scipy.integrate import simpson
import matplotlib.pyplot as plt
import os, glob, re
from pathlib import Path
from datetime import timedelta, datetime
import neurokit2 as nk
from asrpy import ASR
from scipy.signal import spectrogram, butter, filtfilt
import flows
import warnings
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from typing import Tuple

# settings to display all columns
pd.set_option("display.max_columns", None)
# Set the number of rows to display
# pd.set_option('display.max_rows', 50)  # Show up to 100 rows

# General file locations
main_folder = "../Data/"

# Session Overview

In [2]:
# Loading the manually created mapping file
sessions = pd.read_csv(main_folder+"processed/session_overview.csv")
sessions.head(20)

,Session,ID,Config,Folder,Run,Rec_folder
0,2025-07-04-01,bk7zlu,Ear Only,GelledEarOnly/2025-07-04/bk7zlu/,rec1,GelledEarOnly/2025-07-04/bk7zlu/rec1/
1,2025-07-04-01,bk7zlu,Ear Only,GelledEarOnly/2025-07-04/bk7zlu/,rec2,GelledEarOnly/2025-07-04/bk7zlu/rec2/
2,2025-07-04-01,bk7zlu,Ear Only,GelledEarOnly/2025-07-04/bk7zlu/,rec3,GelledEarOnly/2025-07-04/bk7zlu/rec3/
3,2025-07-04-01,aq9b9k,Top+Ears,DryWithHeadband/2025-07-04/aq9b9k/,rec1,DryWithHeadband/2025-07-04/aq9b9k/rec1/
4,2025-07-04-01,aq9b9k,Top+Ears,DryWithHeadband/2025-07-04/aq9b9k/,rec2,DryWithHeadband/2025-07-04/aq9b9k/rec2/
5,2025-07-04-01,aq9b9k,Top+Ears,DryWithHeadband/2025-07-04/aq9b9k/,rec3,DryWithHeadband/2025-07-04/aq9b9k/rec3/
6,2025-07-14-01,fqzqgz,Top+Ears,DryWithHeadband/2025-07-14/fqzqgz/,rec1,DryWithHeadband/2025-07-14/fqzqgz/rec1/
7,2025-07-14-01,fqzqgz,Top+Ears,DryWithHeadband/2025-07-14/fqzqgz/,rec2,DryWithHeadband/2025-07-14/fqzqgz/rec2/
8,2025-07-14-01,fqzqgz,Top+Ears,DryWithHeadband/2025-07-14/fqzqgz/,rec3,DryWithHeadband/2025-07-14/fqzqgz/rec3/
9,2025-07-14-02,7dr6p8,Top+Ears,DryWithHeadband/2025-07-14/7dr6p8/,rec1,DryWithHeadband/2025-07-14/7dr6p8/rec1/


# oTree Logs & Functions

In [3]:
# Load otree log for last sessions
otree_df = pd.read_csv(main_folder + "processed/otree_logs.csv")
otree_df.head(3)

,participant.code,participant._index_in_pages,participant._max_page_index,participant.time_started_utc,participant.token,session.config.name,setup.1.player.headset_color,setup.1.player.token,introduction.1.player.handedness,introduction.1.player.english,introduction.1.player.multilingual,introduction.1.player.age,introduction.1.player.gender,introduction.1.player.glasses,introduction.1.player.eye_sight,introduction.1.player.color_sight,introduction.1.player.hearing,introduction.1.player.occupation,introduction.1.player.education_level,introduction.1.player.headsize,introduction.1.player.weight,introduction.1.player.height,introduction.1.player.hair_style_top,introduction.1.player.hair_style_ears,introduction.1.player.hair_type,introduction.1.player.hair_density,introduction.1.player.beard_style,introduction.1.player.hair_products,introduction.1.player.skin_oily_dry_1,introduction.1.player.skin_oily_dry_2,introduction.1.player.skin_resistant_sensitive_1,introduction.1.player.skin_resistant_sensitive_2,introduction.1.player.skin_products,introduction.1.player.time_since_meal,introduction.1.player.time_since_hydration,introduction.1.player.time_since_caffeine,introduction.1.player.time_since_nicotine,introduction.1.player.time_since_alcohol,introduction.1.player.flow_trait_single,introduction.1.player.flow_trait_sfds1,introduction.1.player.flow_trait_sfds2,introduction.1.player.flow_trait_sfds3,introduction.1.player.flow_trait_sfds4,introduction.1.player.flow_trait_sfds5,introduction.1.player.flow_trait_sfds6,introduction.1.player.flow_trait_sfds7,introduction.1.player.flow_trait_sfds8,introduction.1.player.flow_trait_sfds9,introduction.1.player.flow_trait_sfds10,introduction.1.player.rcsq_sleep,headphones_setup_1.1.player.ux_comfort,headphones_setup_1.1.player.ux_speed,headphones_setup_1.1.player.ux_ease,headphones_setup_1.1.player.ux_look,headphones_setup_1.1.player.wx_public,headphones_setup_1.1.player.wx_private,headphones_setup_1.1.player.wx_conversation,headphones_setup_1.1.player.wx_others,headphones_setup_1.1.player.tlx_single,headphones_setup_1.1.player.sus_01,headphones_setup_1.1.player.sus_02,headphones_setup_1.1.player.sus_03,headphones_setup_1.1.player.sus_04,headphones_setup_1.1.player.sus_05,headphones_setup_1.1.player.sus_06,headphones_setup_1.1.player.sus_07,headphones_setup_1.1.player.sus_08,headphones_setup_1.1.player.sus_09,headphones_setup_1.1.player.sus_10,headphones_setup_1.1.player.mr_mood,headphones_setup_1.1.player.mr_sleepy,headphones_setup_1.1.player.mr_motivy,headphones_setup_1.1.player.mf_single,headphones_setup_1.1.player.mf_01,headphones_setup_1.1.player.mf_02,headphones_setup_1.1.player.mf_03,headphones_setup_1.1.player.mf_04,headphones_setup_1.1.player.rest_actions_eo,headphones_setup_1.1.player.rest_actions_ec,headphones_setup_1.1.subsession.round_number,mathTask_1.1.player.fss06,mathTask_1.1.player.fss08,mathTask_1.1.player.fss09,mathTask_1.1.player.tlx_single,mathTask_1.1.player.difficulty,mathTask_1.1.player.mr_mood,mathTask_1.1.player.mr_sleepy,mathTask_1.1.player.mr_motivy,mathTask_1.1.player.mf_single,mathTask_1.1.player.math_actions,mathTask_1.1.subsession.round_number,mathTask_1.2.player.fss06,mathTask_1.2.player.fss08,mathTask_1.2.player.fss09,mathTask_1.2.player.tlx_single,mathTask_1.2.player.difficulty,mathTask_1.2.player.mr_mood,mathTask_1.2.player.mr_sleepy,mathTask_1.2.player.mr_motivy,mathTask_1.2.player.mf_single,mathTask_1.2.player.math_actions,mathTask_1.2.subsession.round_number,mathTask_1.3.player.fss06,mathTask_1.3.player.fss08,mathTask_1.3.player.fss09,mathTask_1.3.player.tlx_single,mathTask_1.3.player.difficulty,mathTask_1.3.player.mr_mood,mathTask_1.3.player.mr_sleepy,mathTask_1.3.player.mr_motivy,mathTask_1.3.player.mf_single,mathTask_1.3.player.math_actions,mathTask_1.3.subsession.round_number,mathTask_1.4.player.fss06,mathTask_1.4.player.fss08,mathTask_1.4.player.fss09,mathTask_1.4.player.tlx_single,mathTask_1.4.player.difficulty,mathTask_1.4.player.mr_mood,mathTask_1.

In [4]:
# Extract client side timestamps from dedicated otree log columns
def get_actionlog_times(logs_df):
    # Get the timestamps from logging fields
    timestamps = logs_df.filter(regex='actions').dropna(axis=1, how='all').reset_index(drop=True)
    # display(timestamps)
    
    def extract_times(s):
        s = s[0].split(';') # Split string
        s = s[1:] # Remove first, empty entry
        # print(s)

        # Convert into DataFrame
        n_messages = int(len(s)/3)
        messages = []
        ts = []
        ts_unix = []
        for i in range(0, n_messages):
            step = i*3
            messages.append(s[step])
            ts.append(s[step+1])
            ts_unix.append(s[step+2])

        df = pd.DataFrame({'Message':messages, 
                           'TS':ts,
                           'TS_UNIX':ts_unix})

        # Convert unix timestamp to correct datetime
        df['TS'] = pd.to_datetime(pd.to_numeric(df['TS_UNIX'], errors='coerce'), unit='ms', utc=True).dt.tz_convert('Europe/Berlin')
        # print(df.Message.unique())

        return df

    # Parse TS cols
    # TODO: This code could be nicer using a split-apply-combine logic...
    timestamps_parsed = pd.DataFrame()
    for phase in timestamps.columns:
        msgs = extract_times(timestamps[phase])
        msgs['Exp_Phase'] = phase.replace('xTS', '')
        timestamps_parsed = pd.concat([timestamps_parsed, msgs], ignore_index=True) # Had to change this as append is deprecated in pandas since 2.0 -> Alternatively: versioning in requirements.txt 

        
    # Remove unnecessary events
    timestamps_parsed = timestamps_parsed[timestamps_parsed.Message.str.contains("taskStart|taskEnd")]
    # display(timestamps_parsed)
    # display(timestamps_parsed['Exp_Phase'].value_counts())
    
    # Restructure DF
    timestamps_parsed.drop(['TS_UNIX'], axis=1, inplace=True)
    timestamps_parsed['Message'] = timestamps_parsed['Message'].str.replace('task', '', regex=False) # Remove the substring "task"
    timestamps_parsed = timestamps_parsed.pivot(index='Exp_Phase', columns='Message', values='TS').reset_index() # Pivot the DataFrame
    
    timestamps_parsed[['app_name', 'round_number', 'player', 'page_name']] = timestamps_parsed['Exp_Phase'].str.split('.', expand=True)
    timestamps_parsed.drop(['Exp_Phase', 'player'], axis=1, inplace=True)
    timestamps_parsed['token'] = logs_df['participant.token'][0]
    
    # Add mapping to three EEG recordings
    mapping = {
        "_1": "rec1",
        "_2": "rec2",
        "_3": "rec3"
    }

    timestamps_parsed["rec"] = timestamps_parsed["app_name"].str[-2:].map(mapping)
    timestamps_parsed["app_name"] = timestamps_parsed["app_name"].str.replace(r"_[123]$", "", regex=True)
    # display(timestamps_parsed)
    
    # Structure IDs
    timestamps_parsed['page_name'] = timestamps_parsed['page_name'].replace('rest_actions_', '', regex=True)
    timestamps_parsed['page_name'] = timestamps_parsed['page_name'].replace('_actions', '', regex=True)
    timestamps_parsed['Exp_Phase'] = timestamps_parsed['app_name'] + "_" + timestamps_parsed['page_name'] + "_" + timestamps_parsed['round_number']
    
    return timestamps_parsed

# EEG Settings & Functions

In [5]:
# EEG File metdata ------
chans = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8'] # Varies for the two treatments
chans_ear_only = ['A2', 'A3', 'A4', 'A6', 'A7', 'A8']
chans_top_ear = ['A1', 'A2', 'A3', 'A6', 'A7', 'A8']

fs = 250 # Sampling frequency

# PSD parameters
min_freq, max_freq = 1, 45
psd_normalize = False
psd_window_sec, psd_overlap_pct = 1, 0.75

In [6]:
# Label and cut the EEG data
def label_data(eeg, timestamps, cut=True, report=True):        
    # Annotate the data by checking conditions
    for i, row in timestamps.iterrows():        
        idx = (eeg['TS_UNIX'] >= row['Start']) & (eeg['TS_UNIX'] < row['End'])
        eeg.loc[idx, 'Condition'] = row['Exp_Phase']

    # Drop entries where condition is NA
    if cut: eeg = eeg.dropna(subset=['Condition'])
    
    # Check if annotations are correct
    if report:
        display(eeg['Condition'].value_counts())
    
    # print(eeg.Condition.unique())
    return eeg

In [7]:
# Assess ASSR effect across chans
def get_assr_snr(channel_df):
    # Make sure we're working on a copy - to avoid warnings
    channel_df = channel_df.copy()
    
    '''
    # TODO: Should also check Mikkelsen2015 data processing: They exclude some chans without a clear peak at 40Hz
    Mikkelsen2015: Using the ASSR paradigm, we then estimated the signal-to-
    noise ratios (SNR) for both scalp and ear-EEG setups, whereby
    the SNR was defined as the diﬀerence between the logarithm of
    the power at 40 Hz (the signal) and the logarithm of the average
    power in 5 Hz intervals around 40 Hz (the noise floor).
    '''
    # dev = eo_ec_signals[eo_ec_signals.Condition=="headphones_setup_1_eo_1"]["A1"]
    # Split in half
    # TODO: Could also get a more precise indicator of when the ASSR stimulus started - have a timestamp
    first_half, second_half = channel_df.iloc[:channel_df.shape[0]//2], channel_df.iloc[channel_df.shape[0]//2:]   

    return pd.Series({
        "SNR_First_Half": flows.assr_snr(first_half.mV, min_freq=1, max_freq=100, fs=fs, normalize=False, window_sec = 1),
        "SNR_Second_Half": flows.assr_snr(second_half.mV, min_freq=1, max_freq=100, fs=fs, normalize=False, window_sec = 1)
        # log version implemented here:
        # "SNR_First_Half": flows.assr_snr_db(first_half.mV, fs=250.0, f0=40.0, half_width=0.5, flank=4.0, min_f=1.0, max_f=100.0),
        # "SNR_Second_Half": flows.assr_snr_db(first_half.mV, fs=250.0, f0=40.0, half_width=0.5, flank=4.0, min_f=1.0, max_f=100.0),
    })


In [8]:
# Process recordings
def process_recording(active_recording):
    """
    Loop through all recordings
    for index, row in sessions.iterrows():
        process_recording(row)
    """
    # Filter the otree log
    otree_active = otree_df[otree_df['participant.token']==active_recording.ID].reset_index(drop=True)
    # display(otree_active)

    # Extract timings (2 different options)
    timestamps = get_actionlog_times(otree_active)
    # Filter for the active recording
    # timestamps = timestamps[timestamps.rec==active_recording.Run]
    timestamps = timestamps.loc[timestamps['rec'] == active_recording.Run].copy()
    
    # Load EEG data
    rec_folder = main_folder + active_recording.Rec_folder
    eeg, load_report_dict = flows.load_exg_streams_data(rec_folder, chanlist=chans, report=False, cleanFile=True)

    # Now annotate (add condition timings) and cut the EEG Data
    eeg = label_data(eeg, timestamps, cut=True, report=False)
    
    # TODO: Process signals now and extract features of interest
    # TODO: Berger effect during EC/EO rest
    # - So far only visualization
    # - Can use flows.get_channel_band_power() to extract band powers instead
    
    # TODO: ASSR during rest
    # - Not yet checked for correctness...
    # Filter for EC/EO conditions
    wanted = {"headphones_setup_ec_1", "headphones_setup_eo_1"}
    eo_ec_signals = eeg[eeg["Condition"].isin(wanted)]
    # Do for each channel and condition
    # print(eeg['Condition'])  TODO: Wollen wir uns hier nicht eigtl. nur die setups anschauen oder auch "click_task_ec.." ?
    # print( "signals:",  eo_ec_signals)
    eo_ec_signals = eo_ec_signals.drop(['TS_UNIX'], axis=1).melt(id_vars="Condition", value_vars=chans, var_name="Channel", value_name="mV")
    
    snr_df = (
        eo_ec_signals
        .groupby(["Condition", "Channel"])
        .apply(lambda group: get_assr_snr(group), include_groups=False)
        .reset_index()
    )
    snr_df

    # Add the SNR difference to the DataFrame
    snr_df["SNR_Diff"] = snr_df["SNR_Second_Half"] - snr_df["SNR_First_Half"]
    
    # TODO: Other features...
    return eeg, snr_df, load_report_dict

# Process the recordings by looping through the mapping file
# For a single participant
active_recording = sessions.iloc[9]
eeg, snr_df, load_report_dict = process_recording(active_recording)


fig = px.box(
    snr_df,
    x="Channel",
    y="SNR_Diff",
    points="all",  # show individual data points
    color="Channel",  # optional: adds color by channel
    title="SNR Change (Second Half - First Half) by Channel",
    labels={"SNR_Diff": "ΔSNR", "Channel": "EEG Channel"}
)

fig.show()

print('--- All done!\n')

--- All done!



In [9]:
# printing the psd
flows.plot_PSD_two_conditions(eeg, chans, 'headphones_setup_ec_1', 'headphones_setup_eo_1', 
                                fs, min_freq, max_freq, psd_normalize)

In [10]:
# plotting the raw eeg
# flows.plot_eeg_signals(eeg.drop(["Condition", "TS_UNIX"], axis = 1))

___

## Berger Effect
Next we would like to look how prominent we can see the Berger Effect.\
More specifically we will:
1) Look into how the dry electrodes compare to the wet electrodes. 
2) See how different performance metrics will look like as we might not be able to compare results directly -> calculate db magnitude/ relative power
3) Then we will look into intersubject berger effect along sessions. Here an interesting thing to also look into would be a potential alpha peak drift over the sessions 

To do so we will first look into the first run of all participants in wet vs. dry electrodes, build the average psds and compare the results. 

In [11]:
df_first_recs = sessions.query('Run == "rec1" and Config == "Ear Only"')
print(len(df_first_recs))
sessions.head(3)

9


,Session,ID,Config,Folder,Run,Rec_folder
0,2025-07-04-01,bk7zlu,Ear Only,GelledEarOnly/2025-07-04/bk7zlu/,rec1,GelledEarOnly/2025-07-04/bk7zlu/rec1/
1,2025-07-04-01,bk7zlu,Ear Only,GelledEarOnly/2025-07-04/bk7zlu/,rec2,GelledEarOnly/2025-07-04/bk7zlu/rec2/
2,2025-07-04-01,bk7zlu,Ear Only,GelledEarOnly/2025-07-04/bk7zlu/,rec3,GelledEarOnly/2025-07-04/bk7zlu/rec3/


In [12]:
def _extract_run_number(run_value):
    """Return integer run number from strings like 'rec1' or numerics like 1."""
    s = str(run_value).strip().lower()
    digits = re.sub(r"[^0-7]", "", s)
    return int(digits) if digits else None

def compute_session_avg_psd(active_recording, cond_ec_tag, cond_eo_tag):
    """
    For a single session row from `sessions`, load EEG, label EC/EO segments, compute robust PSD per channel,
    and return two DataFrames (EC and EO) with columns ['Frequency','Power'] averaged across channels.
    """

    eeg, snr_df, load_report = process_recording(active_recording)
    fs = float(load_report.get('estimated_fs', 250.0))

    # Select signals for each condition
    ec_df = eeg[eeg['Condition'].astype(str).str.contains(cond_ec_tag, case=False, regex=False)]
    eo_df = eeg[eeg['Condition'].astype(str).str.contains(cond_eo_tag, case=False, regex=False)]

    if ec_df.empty or eo_df.empty:
        raise ValueError(f"Missing EC/EO segments for tags '{cond_ec_tag}'/'{cond_eo_tag}' in recording {active_recording.ID} ({active_recording.Run}).")

    # Average PSD across channels for each condition -> Same building average of delta or delta after average? -> jup

    desired = chans_ear_only if active_recording.Config == "Ear Only" else chans_top_ear
    use_chans = [c for c in desired if c in eeg.columns]
    if not use_chans:
        raise ValueError(
            f"No desired channels found in EEG for {active_recording.ID} ({active_recording.Run}). "
            f"Desired={desired}, available={[c for c in eeg.columns if c.startswith('A')]}"
        )

    def _avg_psd_over_channels(signals_df):
        psd_list = []
        for ch in use_chans:
            series = signals_df[ch]
            psd_df = flows.compute_robust_psd(series, min_freq=min_freq, max_freq=max_freq, fs=fs,
                                              normalize=False, window_sec=2, overlap_pct=0, verbose='WARNING')
            psd_df = psd_df.rename(columns={'Power': f'P_{ch}'})
            psd_list.append(psd_df)
        # merge on frequency
        merged = psd_list[0]
        for nxt in psd_list[1:]:
            merged = merged.merge(nxt, on='Frequency', how='inner')
        merged['Power'] = merged.filter(like='P_').mean(axis=1)
        return merged[['Frequency','Power']]

    ec_psd = _avg_psd_over_channels(ec_df)
    eo_psd = _avg_psd_over_channels(eo_df)

    return ec_psd, eo_psd, fs

def average_psds_across_sessions(df_sessions, config_label, verbose=True): # TODO: Should we exclude dead or too noisy chans/sessions?
    """
    Compute the average EC and EO PSD across all first sessions for a given configuration.
    Returns a dict with keys: 'EC', 'EO', 'alpha_effect', 'n_used', 'fs_median'.
    The alpha_effect is the integrated alpha-band (8-13 Hz) difference EC minus EO (Berger effect magnitude).
    """
    # Filter sessions
    sel = df_sessions.copy()
    sel = sel[sel['Config'].astype(str) == config_label]

    # First-session filter: accept 'rec1' or numeric 1
    run_mask = sel['Run'].astype(str).str.lower().str.contains("rec1") | (sel['Run'].astype(str).str.replace(r"[^0-7]","", regex=True) == "1")
    sel = sel[run_mask]

    if sel.empty:
        raise ValueError(f"No sessions found for config '{config_label}' and Run == 1.")

    ec_psds = []
    eo_psds = []
    fs_list = []

    for row in sel.itertuples(index=False):
        try:
            run_num = _extract_run_number(getattr(row, "Run"))
            cond_ec = f"headphones_setup_ec_{run_num}" if run_num is not None else "headphones_setup_ec_1" # careful with "ec" or only "eo" -> "rec" === True with "ec"
            cond_eo = f"headphones_setup_eo_{run_num}" if run_num is not None else "headphones_setup_eo_1"
            ec_psd, eo_psd, fs = compute_session_avg_psd(row, cond_ec, cond_eo)
            ec_psds.append(ec_psd)
            eo_psds.append(eo_psd)
            fs_list.append(fs)
            if verbose:
                print(f"✅ Processed {row.ID} ({row.Run}) for '{config_label}'")
        except Exception as e:
            warnings.warn(f"Skipping a session due to error: {e}")

    if len(ec_psds) == 0:
        raise RuntimeError(f"All sessions for config '{config_label}' failed to process.")

    # Establish a common frequency grid (use intersection to be safe)
    common_freqs = set(ec_psds[0]['Frequency'].round(6).values)
    for df in ec_psds[1:] + eo_psds:
        common_freqs &= set(df['Frequency'].round(6).values)
    common_freqs = np.array(sorted(list(common_freqs)))
    if common_freqs.size < 10:
        # Fallback: use first EC grid and interpolate others onto it
        base_freqs = ec_psds[0]['Frequency'].values
        def interp_to_base(df):
            return pd.DataFrame({
                'Frequency': base_freqs,
                'Power': np.interp(base_freqs, df['Frequency'].values, df['Power'].values)
            })
        ec_stack = [interp_to_base(df) for df in ec_psds]
        eo_stack = [interp_to_base(df) for df in eo_psds]
    else:
        def trim_to_common(df):
            mask = np.isin(df['Frequency'].round(6).values, common_freqs)
            return df.loc[mask].sort_values('Frequency').reset_index(drop=True)
        ec_stack = [trim_to_common(df) for df in ec_psds]
        eo_stack = [trim_to_common(df) for df in eo_psds]

    # Average across sessions
    ec_avg = pd.concat(ec_stack).groupby('Frequency', as_index=False)['Power'].mean()
    eo_avg = pd.concat(eo_stack).groupby('Frequency', as_index=False)['Power'].mean()

    # Compute Berger effect magnitude = alpha-band (8–13 Hz) area difference EC - EO
    def band_area(psd_df, lo=8.0, hi=13.0):
        m = (psd_df['Frequency'] >= lo) & (psd_df['Frequency'] <= hi)
        if m.sum() < 2:  # not enough points for integration
            return np.nan
        return float(simpson(psd_df.loc[m, 'Power'].values, x=psd_df.loc[m, 'Frequency'].values))

    alpha_ec = band_area(ec_avg, 8.0, 13.0)
    alpha_eo = band_area(eo_avg, 8.0, 13.0)
    alpha_effect = alpha_ec - alpha_eo  # Berger effect: higher alpha in EC than EO

    return {
        'EC': ec_avg,
        'EO': eo_avg,
        'alpha_effect': alpha_effect,
        'n_used': len(ec_stack),
        'fs_median': float(np.median(fs_list)) if fs_list else np.nan,
    }

In [ ]:
# Compute averages
ear_only = average_psds_across_sessions(sessions, config_label="Ear Only", verbose=True)
top_ears = average_psds_across_sessions(sessions, config_label="Top+Ears", verbose=True)

print(f"\nSessions used — Ear Only: {ear_only['n_used']}, Top+Ears: {top_ears['n_used']}")
print(f"Median fs — Ear Only: {ear_only['fs_median']:.2f} Hz, Top+Ears: {top_ears['fs_median']:.2f} Hz")

# Plot averaged PSD curves for EC vs EO per configuration
plt.figure(figsize=(9, 6))
plt.plot(ear_only['EC']['Frequency'], ear_only['EC']['Power'], label="Ear Only — EC")
plt.plot(ear_only['EO']['Frequency'], ear_only['EO']['Power'], label="Ear Only — EO")
plt.plot(top_ears['EC']['Frequency'], top_ears['EC']['Power'], label="Top+Ears — EC")
plt.plot(top_ears['EO']['Frequency'], top_ears['EO']['Power'], label="Top+Ears — EO")
plt.xlim(2, 30)
plt.ylim(0, 500)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.title("Average PSDs (Run 1): EC vs EO per configuration")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Plot Berger effect magnitude (alpha 8–13 Hz: EC - EO) per configuration
configs = ["Ear Only", "Top+Ears"]
effects = [ear_only['alpha_effect'], top_ears['alpha_effect']]

plt.figure(figsize=(6, 4))
plt.bar(configs, effects)
plt.ylabel("Alpha band area (EC - EO)")
plt.title("Berger effect magnitude by configuration (first sessions)")
for i, v in enumerate(effects):
    try:
        plt.text(i, v, f"{v:.3f}", ha='center', va='bottom')
    except Exception:
        pass
plt.grid(True, axis='y', alpha=0.3)
plt.show()


✅ Processed bk7zlu (rec1) for 'Ear Only'
✅ Processed az470i (rec1) for 'Ear Only'
✅ Processed anejys (rec1) for 'Ear Only'
✅ Processed az2bj9 (rec1) for 'Ear Only'
✅ Processed g808pk (rec1) for 'Ear Only'


In [ ]:

# Convert to dB
ear_only_db = {'EC': flows.to_db(ear_only['EC']), 'EO': flows.to_db(ear_only['EO'])}
top_ears_db = {'EC': flows.to_db(top_ears['EC']), 'EO': flows.to_db(top_ears['EO'])}

# Compute absolute, dB, and relative Berger effects (alpha 8–13 Hz)
def alpha_metrics(pair):
    abs_eff = flows.band_area(pair['EC'], 8, 13) - flows.band_area(pair['EO'], 8, 13)
    db_eff  = flows.band_area(flows.to_db(pair['EC']), 8, 13) - flows.band_area(flows.to_db(pair['EO']), 8, 13)
    rel_eff = flows.rel_band_power(pair['EC'], 8, 13) - flows.rel_band_power(pair['EO'], 8, 13)
    return abs_eff, db_eff, rel_eff

eo_abs, eo_db, eo_rel = alpha_metrics(ear_only)
te_abs, te_db, te_rel = alpha_metrics(top_ears)

# Quick broadband scale check (1–30 Hz EC area)
eo_broad = flows.band_area(ear_only['EC'], 1, 30)
te_broad = flows.band_area(top_ears['EC'], 1, 30)

print("=== Summary (first sessions) ===")
print(f"Ear Only:  alpha_abs={eo_abs:.3f}, alpha_dB={eo_db:.3f}, alpha_rel={eo_rel:.4f}, EC_area_1-30Hz={eo_broad:.1f}")
print(f"Top+Ears:  alpha_abs={te_abs:.3f}, alpha_dB={te_db:.3f}, alpha_rel={te_rel:.4f}, EC_area_1-30Hz={te_broad:.1f}")

# Plots in dB and bar charts for the new metrics
# PSDs in dB
plt.figure(figsize=(9, 6))
plt.plot(ear_only_db['EC']['Frequency'], ear_only_db['EC']['Power'], label="Ear Only — EC (dB)")
plt.plot(ear_only_db['EO']['Frequency'], ear_only_db['EO']['Power'], label="Ear Only — EO (dB)")
plt.plot(top_ears_db['EC']['Frequency'], top_ears_db['EC']['Power'], label="Top+Ears — EC (dB)")
plt.plot(top_ears_db['EO']['Frequency'], top_ears_db['EO']['Power'], label="Top+Ears — EO (dB)")
plt.xlim(2, 30)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power (dB)")
plt.title("Average PSDs (Run 1) in dB: EC vs EO per configuration")
plt.legend()
plt.grid(True, axis='y', alpha=0.3)
plt.show()

# Berger effect (alpha area) in dB
plt.figure(figsize=(6, 4))
plt.bar(["Ear Only", "Top+Ears"], [eo_db, te_db])
plt.ylabel("Alpha band area (dB, EC - EO)")
plt.title("Berger effect magnitude (dB)")
for i, v in enumerate([eo_db, te_db]):
    try: plt.text(i, v, f"{v:.2f}", ha='center', va='bottom')
    except: pass
plt.grid(True, axis='y', alpha=0.3)
plt.show()

# Berger effect (relative alpha power)
plt.figure(figsize=(6, 4))
plt.bar(["Ear Only", "Top+Ears"], [eo_rel, te_rel])
plt.ylabel("Δ Relative alpha power (EC - EO)")
plt.title("Berger effect magnitude (relative power)")
for i, v in enumerate([eo_rel, te_rel]):
    try: plt.text(i, v, f"{v:.4f}", ha='center', va='bottom')
    except: pass
plt.grid(True, axis='y', alpha=0.3)
plt.show()


##### Interpretation: 
Okay we can see that intrestingly the absolut berger effect is more visible in the dry electrode setup (even when comparing only electrodes that are present in both setups).

Even when looking at the effect in dB it is more visible in the dry electrodes.\
However, when we look at relative power, this is where we can see the effect more present in the wet electrodes.

___
### ASSR Evaluation: 
We will now look into the Auditory Steady State Response. 
Here multiple things could be interesting: 
1) ASSR within subject over the three sessions: 
2) ASSR between subjects

In [ ]:
target_condition =  "headphones_setup_eo_1" # "headphones_setup_ec_1"
f0 = 40.0
fs_param = fs
win_sec = 5
step_sec = 1.0
snr_thresh = 3.0
max_freq_plot = 100.0
bin_width = 1

for i in range(3):
    active_recording = sessions.iloc[(i+9)]
    eeg, snr_df, load_report_dict = process_recording(active_recording)

    # Build long format
    _tmp = eeg.copy()
    if 'TS_UNIX' in _tmp.columns:
        _tmp = _tmp.drop(columns=['TS_UNIX'])
    eeg_long = pd.melt(_tmp, id_vars=['Condition'], value_vars=chans,
                       var_name='Channel', value_name='mV')

    # Filter by condition for this session
    dfc = eeg_long[eeg_long['Condition'] == target_condition]
    if dfc.empty:
        print(f"Skip {active_recording.ID}: no rows for Condition='{target_condition}'.")
        continue

    # --- Pick a channel for THIS session only ---
    target_channel_session = None
    if isinstance(snr_df, pd.DataFrame) and not snr_df.empty and 'SNR_Second_Half' in snr_df.columns:
        best = snr_df[snr_df['Condition'] == target_condition].sort_values('SNR_Second_Half', ascending=False)
        if not best.empty:
            target_channel_session = best['Channel'].iloc[0]
    if target_channel_session is None:  # fallback to first available channel in this session
        target_channel_session = dfc['Channel'].iloc[0]

    # Extract signal
    sig = dfc.loc[dfc['Channel'] == target_channel_session, 'mV'].to_numpy()
    if sig.size < int(win_sec * fs_param):
        print(f"Skip {active_recording.ID}: signal shorter than window ({sig.size} < {int(win_sec * fs_param)}).")
        continue

    # Scan windows
    scan_df = flows.scan_assr_windows(
        sig, fs=fs_param, win_sec=win_sec, step_sec=step_sec,
        f0=f0, half_width=0.5, flank=4.0
    )

    # Plot SNR over time
    fig_time = flows.plot_assr_snr_over_time(
        scan_df, snr_db_thresh=snr_thresh,
        title=f"ASSR SNR — {target_condition} | {target_channel_session}"
    )
    fig_time.show()

    # --- Robust best-window selection ---
    s = scan_df['snr_db']
    s_nonan = s.dropna()
    if s_nonan.empty:
        print(f"Skip {active_recording.ID}: all SNR values are NaN.")
        continue

    best_idx = s_nonan.idxmax()           # guaranteed not NaN
    best_row = scan_df.loc[best_idx]
    t0 = float(best_row['t_start'])

    # Plot PSD bar for best window
    fig_psd = flows.plot_psd_bar_for_window(
        sig, fs=fs_param, t_start=t0, win_sec=win_sec,
        max_freq=max_freq_plot, bin_width=bin_width, f0=f0
    )
    fig_psd.show()

    print(
        f"Processed {active_recording.ID}: channel '{target_channel_session}', "
        f"window {t0:.2f}-{t0+win_sec:.2f}s; peak SNR={best_row['snr_db']:.2f} dB."
    )



In [ ]:
def _run_to_rec_string(run_value):
    s = str(run_value).strip().lower()
    m = re.search(r"rec\s*([1-3])", s)
    if m:
        return f"rec{m.group(1)}"
    # numeric fallback
    digits = "".join(ch for ch in s if ch.isdigit())
    if digits in {"1","2","3"}:
        return f"rec{digits}"
    return s

# Collect per-session SNR (second half) for EO condition
records = []
for i in range(len(sessions)):
    active_recording = sessions.iloc[i]
    try:
        eeg, snr_df, load_report_dict = process_recording(active_recording)
    except Exception as e:
        print(f"Skip {getattr(active_recording, 'ID', '?')}: process_recording failed: {e}")
        continue

    if not isinstance(snr_df, pd.DataFrame) or snr_df.empty:
        print(f"Skip {getattr(active_recording, 'ID', '?')}: empty snr_df.")
        continue

    if "Condition" not in snr_df.columns or "SNR_Second_Half" not in snr_df.columns:
        print(f"Skip {getattr(active_recording, 'ID', '?')}: required columns missing in snr_df.")
        continue

    # Select EO condition robustly
    mask_eo = snr_df["Condition"].astype(str).str.contains("eo", case=False, na=False)
    snr_eo = snr_df.loc[mask_eo]
    if snr_eo.empty and "headphones_setup_eo_1" in snr_df["Condition"].unique().tolist():
        snr_eo = snr_df[snr_df["Condition"] == "headphones_setup_eo_1"]
    if snr_eo.empty:
        print(f"Skip {getattr(active_recording, 'ID', '?')}: no EO condition rows.")
        continue

    # Single SNR per session: use max across channels (robust)
    val = pd.to_numeric(snr_eo["SNR_Second_Half"], errors="coerce").max()
    if pd.isna(val):
        print(f"Skip {getattr(active_recording, 'ID', '?')}: SNR_Second_Half all NaN.")
        continue

    rec = _run_to_rec_string(getattr(active_recording, 'Run', ''))
    cfg = str(getattr(active_recording, 'Config', ''))
    pid = str(getattr(active_recording, 'ID', ''))
    records.append({"Config": cfg, "ID": pid, "Run": rec, "SNR": float(val)})

df = pd.DataFrame.from_records(records)
if df.empty:
    raise ValueError("No SNR data collected. Ensure sessions/process_recording are working and EO exists.")

# Split by Config (case-insensitive exact match on the two labels used in the notebook)
df_ear = df[df["Config"].str.lower() == "ear only"]
df_top = df[df["Config"].str.lower() == "top+ears"]

run_order = ["rec1", "rec2", "rec3"]

def pivot_per_config(df_cfg):
    if df_cfg.empty:
        return pd.DataFrame(), []
    id_order_cfg = sorted(df_cfg["ID"].unique().tolist())
    mat = df_cfg.pivot_table(index="Run", columns="ID", values="SNR", aggfunc="max")
    # keep only present runs and only IDs of this config (no extraneous grey columns)
    present_runs = [r for r in run_order if r in mat.index]
    mat = mat.reindex(index=present_runs, columns=id_order_cfg)
    return mat, id_order_cfg

mat_ear, ids_ear = pivot_per_config(df_ear)
mat_top, ids_top = pivot_per_config(df_top)

# Shared z-range for comparability (keep if at least one matrix has values)
zvals = []
if not mat_ear.empty: zvals.append(mat_ear.to_numpy(dtype=float))
if not mat_top.empty: zvals.append(mat_top.to_numpy(dtype=float))
if zvals:
    zstack = np.concatenate([v[np.isfinite(v)] for v in [np.ravel(z) for z in zvals] if np.any(np.isfinite(v))])
    zmin = float(np.min(zstack)) if zstack.size else None
    zmax = float(np.max(zstack)) if zstack.size else None
else:
    zmin = zmax = None


fig_ear = None
if not mat_ear.empty:
    fig_ear = px.imshow(
        mat_ear, aspect="auto", origin="lower",
        labels=dict(x="ID", y="Run", color="ASSR SNR (2nd half)"),
        title="ASSR SNR — Ear Only", color_continuous_scale="viridis" # TODO: somehow no viridis doesn't apply -> check why docs says this is fine i guess?
    )
    if zmin is not None and zmax is not None:
        fig_ear.data[0].update(zmin=zmin, zmax=zmax, colorbar=dict(title="ASSR SNR (2nd half)"))

fig_top = None
if not mat_top.empty:
    fig_top = px.imshow(
        mat_top, aspect="auto", origin="lower",
        labels=dict(x="ID", y="Run", color="ASSR SNR (2nd half)"),
        title="ASSR SNR — Top+Ears", color_continuous_scale="viridis" # TODO: somehow no viridis doesn't apply -> check why docs says this is fine i guess?
    )
    if zmin is not None and zmax is not None:
        fig_top.data[0].update(zmin=zmin, zmax=zmax, colorbar=dict(title="ASSR SNR (2nd half)"))

# Compose side-by-side
cols = 0 + (1 if fig_ear is not None else 0) + (1 if fig_top is not None else 0)
if cols == 0:
    raise ValueError("No data to plot for either configuration.")
if cols == 1:
    # Show the single available figure
    (fig_ear or fig_top).show()
else:
    sub = make_subplots(rows=1, cols=2, subplot_titles=("Ear Only", "Top+Ears"))
    sub.add_trace(fig_ear.data[0], row=1, col=1)
    sub.add_trace(fig_top.data[0], row=1, col=2)
    # axes
    sub.update_xaxes(title_text="ID", row=1, col=1)
    sub.update_yaxes(title_text="Run", row=1, col=1)
    sub.update_xaxes(title_text="ID", row=1, col=2)
    sub.update_yaxes(title_text="Run", row=1, col=2)
    # show only right colorbar
    sub.data[0].showscale = False
    sub.data[1].colorbar = dict(title="ASSR SNR (2nd half)")
    sub.update_layout(title_text="ASSR SNR Heatmaps by Configuration", height=500, width=1100)
    sub.show()


___
### RMS Analysis
In the folowing we wil look into the RMS Amplitude change over the course of the 3 recordings, first for each participant.\
Then we will look into the average change for all participants.

In [ ]:
# rms funciton
# RMS aggregate RMS across chosen channels and sessions per config
def rms_uV(x): return np.sqrt(np.mean(np.square(x)))


In [ ]:
rep_chan_amps = flows.report_chan_amps(eeg, report=True)

display(rep_chan_amps)

TypeError: cannot perform __pow__ with this index type: DatetimeArray